In [1]:
import pandas as pd
import numpy as np
from time import time
import math
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import cudf

/ext3/miniconda3/lib/python3.9/site-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


In [2]:
#df_raw = cudf.read_csv('df_train_clean.csv').to_pandas()
df_raw = pd.read_csv('df_train_clean.csv')

In [3]:
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'distance', 'dow', 'month', 'if_night']
df_features = df_raw[features]
df_features['if_night'] = df_features['if_night'].astype('int')

/state/partition1/job-45930720/ipykernel_3668827/3478663700.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['if_night'] = df_features['if_night'].astype('int')


In [4]:
x = df_features.to_numpy()
y = df_raw['fare_amount'].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2784)
x_train = np.column_stack((x_train, np.ones_like(y_train)))
x_test = np.column_stack((x_test, np.ones_like(y_test)))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(42138129, 10) (42138129,) (10534533, 10) (10534533,)


In [5]:
def evaluate(pred, target):
    return np.sqrt(np.mean(((pred - target) ** 2)))

In [6]:
# closed-form solution
def cfs(x_train, y_train, x_test, y_test):
    A = np.linalg.inv(x_train.T.dot(x_train)).dot(x_train.T).dot(y_train)
    pred_train = A @ x_train.T
    pred_test = A @ x_test.T
    return A, pred_train, pred_test

A, pred_train, pred_test = cfs(x_train, y_train, x_test, y_test)    
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))

train RMSE: 3.888225323375795, test RMSE: 3.886347469446173


### Evaluation

**numpy**

In [7]:
%timeit -n 10 evaluate(pred_train, y_train)

135 ms ± 3.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**sklearn**

In [8]:
from sklearn.metrics import mean_squared_error
%timeit -n 10 mean_squared_error(pred_train, y_train, squared=False)

187 ms ± 4.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Numba**

In [9]:
from numba import jit

@jit(nopython=True, parallel = True)
def numba_evaluate(pred, target):
    return np.sqrt(np.mean(((pred - target) ** 2)))

%timeit -n 10 numba_evaluate(pred_train, y_train)

/ext3/miniconda3/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


The slowest run took 38.96 times longer than the fastest. This could mean that an intermediate result is being cached.
41.9 ms ± 86.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Cython**

In [10]:
%load_ext Cython

In [11]:
%%cython --annotate
import numpy as np
cimport numpy as np
def cython_evaluate(np.ndarray[np.double_t, ndim=1] pred, np.ndarray[np.double_t, ndim=1] target):
    # cdef double rmse
    # rmse = np.sqrt(np.mean(((pred - target) ** 2)))
    # return rmse
    cdef double mse = 0.0
    cdef double diff
    cdef int i
        
    for i in range(pred.shape[0]):
        diff = pred[i] - target[i]
        mse = mse + diff * diff
    
    return np.sqrt(mse / pred.shape[0])

In [12]:
%timeit -n 10 cython_evaluate(pred_train, y_train)

63.1 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Closed-form Solution

**numpy**

In [13]:
# closed-form solution
def cfs(x_train, y_train, x_test, y_test):
    A = np.linalg.inv(x_train.T.dot(x_train)).dot(x_train.T).dot(y_train)
    pred_train = A @ x_train.T
    pred_test = A @ x_test.T
    return A, pred_train, pred_test

A, pred_train, pred_test = cfs(x_train, y_train, x_test, y_test)
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))

%timeit -n 1 cfs(x_train, y_train, x_test, y_test)

train RMSE: 3.888225323375795, test RMSE: 3.886347469446173
1.39 s ± 81.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Numba**

In [14]:
from numba import jit

@jit(nopython=True, parallel = True)
def numba_cfs(x_train, y_train, x_test, y_test):
    A = np.linalg.inv(x_train.T.dot(x_train)).dot(x_train.T).dot(y_train)
    pred_train = A @ x_train.T
    pred_test = A @ x_test.T
    return A, pred_train, pred_test
    
A, pred_train, pred_test = numba_cfs(x_train, y_train, x_test, y_test)
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))    

%timeit -n 1 numba_cfs(x_train, y_train, x_test, y_test)

/ext3/miniconda3/lib/python3.9/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../state/partition1/job-45930720/ipykernel_3668827/3453340879.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


train RMSE: 3.8882253233757824, test RMSE: 3.886347469400136
678 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Cython**

In [15]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [16]:
%%cython --annotate
import numpy as np
cimport numpy as np

def cython_cfs(np.ndarray[np.float64_t, ndim=2] x_train, np.ndarray[np.float64_t, ndim=1] y_train, np.ndarray[np.float64_t, ndim=2] x_test, np.ndarray[np.float64_t, ndim=1] y_test):
    
    cdef np.ndarray[np.float64_t, ndim=1] A
    cdef np.ndarray[np.float64_t, ndim=1] pred_train, pred_test
    
    A = np.linalg.inv(np.dot(x_train.T, x_train)).dot(x_train.T).dot(y_train)
    pred_train = np.dot(A, x_train.T)
    pred_test = np.dot(A, x_test.T)
    
    return A, pred_train, pred_test

In [17]:
A, pred_train, pred_test = cython_cfs(x_train, y_train, x_test, y_test)
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))

%timeit -n 1 cython_cfs(x_train, y_train, x_test, y_test)

train RMSE: 3.888225323375795, test RMSE: 3.886347469446173
1.4 s ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**sklearn.linear_model**

In [18]:
from sklearn.linear_model import LinearRegression
def skl(x_train, y_train, x_test, y_test):
    model = LinearRegression().fit(x_train, y_train)
    pred_train = model.predict(x_train)
    pred_test = model.predict(x_test)
    return model, pred_train, pred_test
    
A, pred_train, pred_test = skl(x_train, y_train, x_test, y_test)
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))

%timeit -n 1 skl(x_train, y_train, x_test, y_test)

train RMSE: 3.888225323375789, test RMSE: 3.8863474693983577
11.8 s ± 208 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**statsmodels.OLS**

In [19]:
import statsmodels.api as sm
def statsmodels(x_train, y_train, x_test, y_test):
    model = sm.OLS(y_train, x_train).fit()
    pred_train = model.predict(x_train)
    pred_test = model.predict(x_test)
    return model, pred_train, pred_test
    
A, pred_train, pred_test = statsmodels(x_train, y_train, x_test, y_test)
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))

%timeit -n 1 statsmodels(x_train, y_train, x_test, y_test)

train RMSE: 3.8882253233757877, test RMSE: 3.8863474693983586
12.9 s ± 604 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**numpy QR decomposition**

In [20]:
from numpy.linalg import qr, inv, lstsq
def QR(x_train, y_train, x_test, y_test):
    Q, R = qr(x_train)
    A = inv(R).dot(Q.T).dot(y_train)
    pred_train = A @ x_train.T
    pred_test = A @ x_test.T
    return A, pred_train, pred_test
    
A, pred_train, pred_test = QR(x_train, y_train, x_test, y_test)
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))

%timeit -n 1 QR(x_train, y_train, x_test, y_test)

train RMSE: 3.888225323375789, test RMSE: 3.8863474693983586
8.72 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**numpy lstdq**

In [21]:
from numpy.linalg import qr, inv, lstsq
def nplstsq(x_train, y_train, x_test, y_test):
    A, _, _, _ = lstsq(x_train, y_train, rcond=None)
    pred_train = A @ x_train.T
    pred_test = A @ x_test.T
    return A, pred_train, pred_test
    
A, pred_train, pred_test = nplstsq(x_train, y_train, x_test, y_test)
print('train RMSE: {}, test RMSE: {}'.format(evaluate(pred_train, y_train), evaluate(pred_test, y_test)))

%timeit -n 1 nplstsq(x_train, y_train, x_test, y_test)

train RMSE: 3.8882253233757886, test RMSE: 3.8863474693983595
3.4 s ± 290 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Putting together - optimal closed-form solution**

In [22]:
from time import time
start_time = time()
def f():
    A, pred_train, pred_test = numba_cfs(x_train, y_train, x_test, y_test)
    numba_evaluate(pred_train, y_train)
%timeit -n 10 f

32.5 ns ± 9.29 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Gradeint Descent

**numpy**

In [23]:
import numpy as np

def gd(x, y, alpha=0.0001, iterations=10):
    m, n = x.shape
    theta = np.random.rand(n)
    for i in range(iterations):
        gradient = (1 / m) * x.T.dot(x.dot(theta) - y)
        theta -= alpha * gradient
    return theta

In [24]:
start_time = time()
A = gd(x_train, y_train)
print(time()-start_time)

pred_train = x_train.dot(A)
pred_test = x_test.dot(A)
numba_evaluate(pred_train, y_train)

2.445025682449341


5.879738758886684

In [25]:
%timeit -n 10 gd(x_train, y_train)

2.28 s ± 43.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Numba**

In [26]:
from numba import jit
from time import time
@jit(nopython=True, parallel = True)
def numba_gd(x, y, alpha=0.0001, iterations=10):
    m, n = x.shape
    theta = np.random.rand(n)
    for i in range(iterations):
        gradient = (1 / m) * x.T.dot(x.dot(theta) - y)
        theta -= alpha * gradient
    return theta

In [27]:
start_time = time()
A = numba_gd(x_train, y_train)
print(time()-start_time)

pred_train = x_train.dot(A)
pred_test = x_test.dot(A)
numba_evaluate(pred_train, y_train)

3.8619813919067383


5.626023290905243

In [28]:
%timeit -n 10 numba_gd(x_train, y_train)

1.32 s ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Cython**

In [29]:
%%cython --annotate
import numpy as np
cimport numpy as np
from cython.parallel import prange
import time

def cython_gd(np.ndarray[np.float64_t, ndim=2] x, np.ndarray[np.float64_t, ndim=1] y, double alpha=0.0001, int iterations=10):
    cdef int m = x.shape[0]
    cdef int n = x.shape[1]
    cdef np.ndarray[np.float64_t, ndim=1] theta = np.random.rand(n)
    cdef np.ndarray[np.float64_t, ndim=1] gradient = np.empty(n)
    cdef int i, j

    for _ in range(iterations):
        for j in prange(n, nogil=True):
            gradient[j] = 0.0
            for i in range(m):
                gradient[j] += (x[i, j] * theta[j]) - y[i]
            gradient[j] /= m
            theta[j] -= alpha * gradient[j]

    return theta

In [30]:
start_time = time.time()
A = cython_gd(x_train, y_train)
print(time.time()-start_time)

pred_train = np.dot(x_train, A)
pred_test = np.dot(x_test, A)
numba_evaluate(pred_train, y_train)

29.148980379104614


78.92022728910284

In [31]:
%timeit -n 1 cython_gd(x_train, y_train)

27.5 s ± 598 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Minibatch Gradient Descent

**numpy**

In [32]:
def mb(x, y, batch_size=64, learning_rate=0.0001, epochs=3):
    m, n = x.shape
    theta = np.random.randn(n)
    for epoch in tqdm(range(epochs)):
        shuffled_indices = np.random.permutation(m)
        x_shuffled = x[shuffled_indices]
        y_shuffled = y[shuffled_indices]
        
        for i in range(0, m, batch_size):
            xi = x_shuffled[i:i+batch_size]
            yi = y_shuffled[i:i+batch_size]
            gradients = 1/batch_size * xi.T.dot(xi.dot(theta) - yi)
            theta = theta - learning_rate * gradients
    return theta

In [33]:
A = mb(x_train, y_train)
pred_train = x_train.dot(A)
pred_test = x_test.dot(A)
numba_evaluate(pred_train, y_train)

100%|██████████| 3/3 [00:31<00:00, 10.60s/it]


3.9966263052839683

In [34]:
%timeit -n 1 mb(x_train, y_train)

100%|██████████| 3/3 [00:31<00:00, 10.53s/it]

31.8 s ± 274 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Numba**

In [35]:
from numba import jit

@jit(nopython=True, parallel = True)
def numba_mb(x, y, batch_size=64, learning_rate=0.0001, epochs=3):
    m, n = x.shape
    theta = np.random.randn(n)
    for epoch in range(epochs):
        shuffled_indices = np.random.permutation(m)
        x_shuffled = x[shuffled_indices]
        y_shuffled = y[shuffled_indices]
        
        for i in range(0, m, batch_size):
            xi = x_shuffled[i:i+batch_size]
            yi = y_shuffled[i:i+batch_size]
            gradients = 1/batch_size * xi.T.dot(xi.dot(theta) - yi)
            theta = theta - learning_rate * gradients
    return theta

In [36]:
import numpy as np
from numba import jit

@jit(nopython=True)
def numba_mb(x, y, batch_size=64, learning_rate=0.0001, epochs=3):
    m, n = x.shape
    theta = np.random.randn(n)
    for epoch in range(epochs):
        shuffled_indices = np.random.permutation(m)
        x_shuffled = x[shuffled_indices]
        y_shuffled = y[shuffled_indices]
        
        for i in range(0, m, batch_size):
            xi = x_shuffled[i:i+batch_size]
            yi = y_shuffled[i:i+batch_size]
            gradients = np.zeros(n)
            for j in range(len(xi)):
                dot_product = 0.0
                for k in range(n):
                    dot_product += xi[j, k] * theta[k]
                error = dot_product - yi[j]
                for k in range(n):
                    gradients[k] += xi[j, k] * error
            for k in range(n):
                gradients[k] /= batch_size
            for k in range(n):
                theta[k] -= learning_rate * gradients[k]
    return theta

In [37]:
from time import time
start_time = time()
A = numba_mb(x_train, y_train)
pred_train = x_train.dot(A)
pred_test = x_test.dot(A)
print(time()-start_time)
numba_evaluate(pred_train, y_train)

24.046466827392578


3.94894645169484

In [38]:
%timeit -n 1 numba_mb(x_train, y_train)

21.9 s ± 218 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Cython**

In [39]:
%%cython --annotate
import numpy as np
cimport numpy as np

def cython_mb(np.ndarray[np.float64_t, ndim=2] x, np.ndarray[np.float64_t, ndim=1] y, int batch_size=64, float learning_rate=0.0001, int epochs=3):
    cdef int m = x.shape[0]
    cdef int n = x.shape[1]
    cdef np.ndarray[np.float64_t, ndim=1] theta = np.random.randn(n)
    cdef np.ndarray[np.float64_t, ndim=2] x_shuffled
    cdef np.ndarray[np.float64_t, ndim=1] y_shuffled
    cdef np.ndarray[np.float64_t, ndim=2] xi
    cdef np.ndarray[np.float64_t, ndim=1] yi
    cdef np.ndarray[np.float64_t, ndim=1] gradients
    
    for epoch in range(epochs):
        shuffled_indices = np.random.permutation(m)
        x_shuffled = x[shuffled_indices]
        y_shuffled = y[shuffled_indices]
        
        for i in range(0, m, batch_size):
            xi = x_shuffled[i:i+batch_size]
            yi = y_shuffled[i:i+batch_size]
            gradients = 1.0 / batch_size * xi.T.dot(xi.dot(theta) - yi)
            theta = theta - learning_rate * gradients
    return theta

In [40]:
%timeit -n 1 cython_mb(x_train, y_train)

32.8 s ± 192 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**SGD**

In [41]:
def sgd(x, y, learning_rate=0.0001, epochs=1):
    m, n = x.shape
    theta = np.random.randn(n)
    for epoch in tqdm(range(epochs)):
        for i in range(m):
            gradient = x[i].T.dot(x[i].dot(theta) - y[i])
            theta = theta - learning_rate * gradient
    return theta

In [42]:
A = sgd(x_train, y_train, epochs=1)
pred_train = x_train.dot(A)
pred_test = x_test.dot(A)
numba_evaluate(pred_train, y_train)

100%|██████████| 1/1 [02:33<00:00, 153.29s/it]


5.000296695533813

In [43]:
from numba import jit

@jit(nopython=True, parallel = True)
def numba_sgd(x, y, learning_rate=0.0001, epochs=1):
    m, n = x.shape
    theta = np.random.randn(n)
    for epoch in range(epochs):
        for i in range(m):
            gradient = x[i].T.dot(x[i].dot(theta) - y[i])
            theta = theta - learning_rate * gradient
    return theta

In [44]:
start_time = time()
A = numba_mb(x_train, y_train, 1, learning_rate=0.0001, epochs=1)
pred_train = x_train.dot(A)
pred_test = x_test.dot(A)
print(time()-start_time)
numba_evaluate(pred_train, y_train)

10.912972450256348


4.736041487503243